In [1]:
import spacy

In [17]:
nlp = spacy.load("en_core_web_sm")

In [18]:
texts = [
    "Net income was $9.4 million compares to the prior year of $2.7 million.",
    "Revenue exceeded twelve asking billion dollars, with a loss of $1b.",
]

In [19]:
docs = [nlp(text) for text in texts]

In [21]:
for token in docs[0]:
    print(token.text, token.lemma_)

Net net
income income
was be
$ $
9.4 9.4
million million
compares compare
to to
the the
prior prior
year year
of of
$ $
2.7 2.7
million million
. .


In [23]:
u'test'

'test'

In [24]:
r"test"

'test'

In [25]:
r'test'

'test'